In [1]:
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter
from Bio import PDB, Seq, pairwise2
import pandas as pd
import numpy as np
import subprocess
import sys
import shutil
import os
import gzip
from tqdm import tqdm
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from openfold_light.residue_constants import restype_3to1, restype_1to3 # map 3 letters amino acid code to 1 letter code
from cg import cg_dict

/cloud-home/U1034010/.magellan/conda/envs/nanofold/lib/python3.9/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


# Read from parsed VH sequences

In [2]:
csvfp="./master_pdb_parsed_v2_curated_v4_multichain_Ag_filtered_v2_separated_all_VH_complexes_unique_seq_level_with_seq.csv"
df = pd.read_csv(csvfp)

# get all the files names from "Name"
names = df["Name"].str.split('>').str[0].to_list() # e.g. "7lx5-assembly1" or "5ku2" 
# df["Name"] = df["Name"].str.split('>').str[0]
subchain_id = df["Chain_VH"].to_list() # e.g. "B_0" or "B-2_0"
sequence_vh = df["Sequence_VH"].to_list()


# Download all raw the files 

In [3]:
# save target filenames
outfp = "./pdb_1700_raw"
with open("./cache_filenames.txt", "w") as file: 
    file.write(", ".join(names))
os.makedirs(outfp, exist_ok=True)

# change permission 
subprocess.run(["chmod", "+x", "./batch_download.sh"], check=True)

# run the shell script 
subprocess.run(["bash", "./batch_download.sh", "-f", "./cache_filenames.txt", "-o", outfp, "-c"], stdout=subprocess.DEVNULL)

# delete cached files names 
os.remove("./cache_filenames.txt")

# gzip decompress all files 
for fp in tqdm(os.listdir(outfp), desc="Decompress .gz files..."):
    with gzip.open(os.path.join(outfp, fp), "rb") as f_in, \
        open(os.path.join(outfp, fp.strip(".gz")), "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)
    # delete gzip files 
    os.remove(os.path.join(outfp, fp))

Decompress .gz files...: 100%|██████████| 1412/1412 [00:06<00:00, 216.26it/s]


# Extract entire target VH chain from raw files
- then record all the coordinates of the corresponding amino acid in given sequence 
- non-existing coordinates are set as nan 
- amino acid that doesn't exist in the curated sequence, but appear to be unknown in the .cif record, should be X
- save the result .cif files in `pdb_1700`

In [4]:
def parse_structure(subcid, curated_seq, cid, aligned_sequences):
    # parse the target file 
    parser = PDB.MMCIFParser(QUIET=True)
    try: # 7evw-assembly1 isn't downloaded so skip with try-except
        structure = parser.get_structure(name, os.path.join(outfp, name+".cif"))
        model = structure[0]

        # extract chain 
        chain = model[cid]    

        # extract both residue objects and their string sequences
        chain_residues = [(restype_3to1[residue.get_resname()], residue) for residue in chain if residue.get_resname() in restype_3to1]
        chain_seq = ''.join([restype_3to1[residue.get_resname()] for residue in chain if residue.get_resname() in restype_3to1])


        # align curacted seq with chain seq
        if chain_seq != curated_seq: 

            # note on parameters: 
            # [seq1, seq2, 
            #  reward given to identical character,
            #  deduction for non-identical character, 
            #  deduction for open gap,
            #  deduction for extending sequence]
            alignment = pairwise2.align.globalms(chain_seq, curated_seq, 5, -3, -1., -.5, gap_char='X')[0]

            # EX: 
            # len(seqA) == len(seqB)
            # chain_seq   = EVQLQESGGGLVYDSLRLSCASSRSIDGINIMRWYRQAPGKQRGMVAVVTGWGSTNYVDSVKGRFIISRDSAKDTVYLQMNNLKPEDTAVYSCNAIYRGSEYWGQGTQVTVS 
            # curated_seq = ESGEMLFTVKKDSLRLSCASSRSIDGINIMRWYRQAPGKQRGMVAVVTGWGSTNYVDSVKGRFIISRDSAKDTVYLQMNNLKPEDTAVYSCNAIYRGSEYWGQGTQVTVS 
            # seqA = EVQLQESGGGXXLXXVYXXDSLRLSCASSRSIDGINIMRWYRQAPGKQRGMVAVVTGWGSTNYVDSVKGRFIISRDSAKDTVYLQMNNLKPEDTAVYSCNAIYRGSEYWGQGTQVTVS
            # seqB = XXXXXESXXGEMLFTVXKKDSLRLSCASSRSIDGINIMRWYRQAPGKQRGMVAVVTGWGSTNYVDSVKGRFIISRDSAKDTVYLQMNNLKPEDTAVYSCNAIYRGSEYWGQGTQVTVS
            aligned_chain_seq = alignment.seqA
            aligned_curated_seq = alignment.seqB

            # Need to re-index the chain_residues according to the aligned chain sequence
            aligned_chain_residues = []
            chain_indx = 0
            for aligned_indx, aligned_restype in enumerate(aligned_chain_seq):
                # (1) inserted unknown residue
                if aligned_restype == 'X':
                    aligned_chain_residues.append((aligned_restype, None)) # No residue assigned to X, will create a new one later 
                # (2) keep original chain residue 
                else: 
                    chain_restype, chain_residue = chain_residues[chain_indx]
                    aligned_chain_residues.append((aligned_restype, chain_residue))
                    chain_indx += 1    

        else: 
            aligned_chain_seq = chain_seq 
            aligned_chain_residues = chain_residues
            aligned_curated_seq = curated_seq


        # construct new structure
        new_structure = PDB.Structure.Structure(id=name[:4])
        new_model = PDB.Model.Model(id=model.id)
        new_chain = PDB.Chain.Chain(id=cid)

        # insert aligned residue according to aligned curated sequence  
        rid = 1 # new residue id 
        for ri, curated_restype in enumerate(aligned_curated_seq): 
            chain_residue = aligned_chain_residues[ri][1]
            chain_restype = aligned_chain_seq[ri]

            # if residue is X or unknown, ignore the residue in the original protein
            if curated_restype == 'X' or curated_restype not in restype_1to3: 
                continue
            # if residue is the same as the original protein, copy the residue
            elif curated_restype == chain_restype:
                new_residue = PDB.Residue.Residue((' ', rid, ' '), restype_1to3[curated_restype], chain_residue.segid)

                # copy the atoms
                for atom in chain_residue:
                    new_residue.add(atom.copy()) 

                # add residue to the chain 
                new_chain.add(new_residue)
                rid += 1
            # if residue is different from the original protein, create a new residue with the curated residue type
            else:
                assert chain_restype == 'X', "Chain residue is not X when curated residue is different but not X."
                new_residue = PDB.Residue.Residue((' ', rid, ' '), restype_1to3[curated_restype], "")

                # choose the first atoms assignment from CG group as placeholder
                # TODO: change this to the average of the neighboring atoms
                coord = np.array([np.nan, np.nan, np.nan]) 
                for serial_num, atom_name in enumerate(cg_dict[restype_1to3[curated_restype]][0]):
                    new_atom = PDB.Atom.Atom(name=atom_name, 
                                            coord=coord, 
                                            bfactor=0.0, 
                                            occupancy=1.0, 
                                            altloc=" ",
                                            fullname=f" {atom_name} ", 
                                            serial_number=serial_num,
                                            element=atom_name[0])
                    new_residue.add(new_atom)

                new_chain.add(new_residue)
                rid += 1

        # add new chain to model and model to structure 
        new_model.add(new_chain)
        new_structure.add(new_model)
        
        # save aligned seq to a list 
        aligned_sequences.append(aligned_curated_seq)
        
        # write the curated structure to file 
        cifio = PDB.mmcifio.MMCIFIO()
        cifio.set_structure(new_structure)
        cifio.save(destfp+f'/{name}_{subcid}.cif')

    except FileNotFoundError as e: 
        tqdm.write(f"\nFileNotFound: {name}")

    

outfp = "./pdb_1700_raw"
destfp = "./pdb_1700"
os.makedirs(destfp, exist_ok=True)
count_diff = 0 # 1252 (729 without X) chain sequence is different from the subchains
aligned_sequences = []


# TODO: Improve runtime to use multiprocessing
pbar = tqdm(names)
for indx, name in enumerate(pbar): 
    # get the corresponding chain and sequence
    subcid = subchain_id[indx] # e.g. K_0
    curated_seq = sequence_vh[indx] # e.g. QVQLQESGGGLV...
    cid, subid = subcid.split('_')
    parse_structure(subcid, curated_seq, cid, aligned_sequences)



  0%|          | 0/1709 [00:00<?, ?it/s]

  7%|▋         | 114/1709 [00:45<10:16,  2.59it/s]


FileNotFound: 7evw-assembly1


100%|██████████| 1709/1709 [13:24<00:00,  2.12it/s]


# Rename the csv columns 
* From Chain_VH,Name,Sequence_VH
* To uid,seq,chain_id

In [5]:
df = pd.read_csv("./master_pdb_parsed_v2_curated_v4_multichain_Ag_filtered_v2_separated_all_VH_complexes_unique_seq_level_with_seq.csv")
df = df.drop(df.loc[df["Name"].str.contains("7evw-assembly1")].index).reset_index(drop=True) # this one is not working 

df = df.rename(columns={
                    "Chain_VH": "chain_id",
                    "Name": "uid",
                    "Sequence_VH": "seq",
                })
df["uid"] = df["uid"].str.split(">").str[0]
df.head()

,chain_id,uid,seq
0,A_0,7lx5-assembly1,QVQLQESGGGLVQPGGSLRLSCAASGFTFRRYLMGWARQVPGKGLE...
1,N_0,6p9y-assembly1,QVQLQESGGGLVQPGGSLRLSCAASGFTFSNYKMNWVRQAPGKGLE...
2,E_0,4w2q-assembly3,KVQLQESGGGLVQVGGSLRLSCKASGFTFRSSAMGWYRRAPGKQRE...
3,C_0,4x7e-assembly1,DVQLVESGGGLVQPGGSLRLSCAASGSIFSIYAMGWYRQAPGKQRE...
4,B_0,7x2m-assembly1,QVQLQESGGGLVQPGGSLRLSCAASGDTLDLYAIGWFRQTPGEERE...


In [ ]:
# sanity check: shouldn't be any X in the seq 
# df['seq'].str.contains('X').sum() == 0

True

In [6]:
df.to_csv("./nanobody_1700_unique_seq.csv", index=None)

In [6]:
# # remove nanobody entries that don't exist
# print(df.shape)
# targets = set(os.listdir("./pdb_1700"))
# keeprows = df.apply(lambda r: str(r["uid"] + '_' + r['chain_id'] + ".cif") in targets, axis=1)
# print(keeprows.sum())
# df = df.loc[keeprows]
# print("|-----> ", df.shape) # Weird that the numbers doesn't change here...

(1708, 4)
1708
|----->  (1708, 4)


In [7]:
# df.to_csv("./nanobody_1700_unique_seq.csv", index=None)

In [9]:
# # remove trailing integer on chain_id (A_0 --> A)
# df1 = pd.read_csv("./master_pdb_parsed_v2_curated_v4_multichain_Ag_filtered_v2_separated_all_VH_complexes_unique_seq_level_with_seq.csv")
# df1 = df1.drop(df1.loc[df1["Name"].str.contains("7evw-assembly1")].index).reset_index(drop=True)
# df1 = df1.rename(columns={
#                     "Chain_VH": "chain_id",
#                     "Name": "uid",
#                     "Sequence_VH": "original_seq",
#                 })


# df2 = pd.read_csv("./nanobody_1700_unique_seq.csv")
# df2["chain_id"] = df1["chain_id"]
# df2["seq"] = df1["original_seq"]
# df2.to_csv("./nanobody_1700_unique_seq.csv", index=None)

# Clean up raw file to save space

In [9]:
# Clean up all downloaded raw files 
# outfp = "./pdb_1700_raw"
# shutil.rmtree(outfp)